In [118]:
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering


In [119]:

# Charger les données
df = pd.read_csv("/home/codespace/project_data/dataVisualization/result.csv")

# Regrouper les catégories avec peu de données dans une catégorie 'Other'
df['bug type'] = df['bug type'].replace(['Butterfly', 'Hover fly', 'Wasp'], 'Other')

# Sélectionner les colonnes de caractéristiques et la cible
train_predictor_columns = ["Median_R", "Median_G", "Median_B", "Std_R", "Std_G", "Std_B", "Area"]
target_labels = df["bug type"]
train_feats = df[train_predictor_columns]

# Supprimer les valeurs manquantes
df = df.dropna()

# Imputation des valeurs manquantes
imputer = SimpleImputer(strategy='mean')
train_feats = imputer.fit_transform(train_feats)

# Normalisation des données
scaler = StandardScaler()
train_feats_scaled = scaler.fit_transform(train_feats)

# Appliquer KMeans
kmeans = KMeans(n_clusters=3, random_state=0)  # Vous pouvez ajuster n_clusters selon vos résultats précédents
kmeans_labels = kmeans.fit_predict(train_feats_scaled)

# Appliquer Agglomerative Clustering
agg_clustering = AgglomerativeClustering(n_clusters=3)  # Vous pouvez ajuster n_clusters selon vos résultats précédents
agg_labels = agg_clustering.fit_predict(train_feats_scaled)

# Ajouter les étiquettes de clusters comme caractéristiques supplémentaires
train_feats_with_clusters = pd.DataFrame(train_feats, columns=train_predictor_columns)
train_feats_with_clusters['KMeans_Cluster'] = kmeans_labels
train_feats_with_clusters['Agg_Cluster'] = agg_labels

# Division des données en ensembles d'entraînement et de test
x_train, x_test, y_train, y_test = train_test_split(train_feats_with_clusters, target_labels, test_size=0.2, random_state=0)

# Appliquer KNN
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)
y_pred = knn.predict(x_test)

# Évaluation de la performance
classification_report = metrics.classification_report(y_test, y_pred)
print("Classification Report:")
print(classification_report)



Classification Report:
              precision    recall  f1-score   support

         Bee       0.50      0.57      0.53        21
   Bumblebee       0.62      0.60      0.61        25
       Other       0.50      0.25      0.33         4

    accuracy                           0.56        50
   macro avg       0.54      0.47      0.49        50
weighted avg       0.56      0.56      0.56        50



In [120]:
# Isoler les lignes de result.csv qui ont été utilisées pour tester le modèle
test_df = df.loc[x_test.index]
test_df.drop(train_predictor_columns, axis=1, inplace=True)
test_df.drop(['sym_index', 'species'], axis=1, inplace=True)

test_df['Predicted_Bug_Type'] = y_pred

test_df['Recognition'] = test_df['bug type'] == test_df['Predicted_Bug_Type']
test_df.to_csv("/home/codespace/project_data/MachineLearning/new_results.csv", index=False)